# 따릉이 프로젝트
서울 열린데이터 광장에서 제공하는 서울시 공공자전거 데이터
이용하여 각종 데이터 분석 및 시각화

**DataSet

Data1 : 서울특별시 공공자전거 대여소 정보  
서울특별시 공공자전거 대여소(따릉이) 현황정보입니다.  
http://data.seoul.go.kr/dataList/OA-13252/F/1/datasetView.do

Data2 : 서울시 공공자전거 이용현황  
서울시에서 운영하는 공공자전거(따릉이)에 대한 기간별, 대여소별 이용(대여, 반납)현황 데이터입니다.  
http://data.seoul.go.kr/dataList/OA-14994/F/1/datasetView.do

Data3 : 서울시 공공자전거 대여소별 이용정보(월별)  
대여일시, 대여시간, 대여소번호, 대여소명, 정기권유무, 성별, 연령대, 탄소량, 이동거리, 이동시간 정보를 제공합니다.  
http://data.seoul.go.kr/dataList/OA-15248/F/1/datasetView.do

In [541]:
#각종 패키지 불러오기
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import seaborn as sns
font_path = './NanumGothic.ttf' # 나눔고딕폰트 사용
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False #minus 폰트깨짐 방지

plt.style.use('seaborn')
sns.set(font="NanumGothic", 
rc={"axes.unicode_minus":False}) #seaborn 한글폰트사용

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [542]:
#데이터불러오기 
#cp949 인코딩 사용 한글 데이터 커버
Data1 = pd.read_csv('data/공공자전거 대여소 정보(21.01.31 기준).csv', encoding = 'cp949')
Data2 = pd.read_csv('data/대여소별 이용정보(21.7-21.12).csv', encoding = 'cp949')
Data3 = pd.read_csv('data/서울특별시 공공자전거 이용정보(월별)_21.12.csv', encoding = 'cp949')

In [543]:
#데이터 확인하기
Data2.head()

,대여소 그룹,대여소 명,대여 일자 / 월,대여 건수
0,강남구,2301. 현대고등학교 건너편,202107,2545
1,강남구,2302. 교보타워 버스정류장(신논현역 3번출구 후면),202107,1176
2,강남구,2303. 논현역 7번출구,202107,1467
3,강남구,2304. 신영 ROYAL PALACE 앞,202107,349
4,강남구,2305. MCM 본사 직영점 앞,202107,341


In [544]:
Data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102926 entries, 0 to 102925
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   대여일자     102926 non-null  object 
 1   대여소번호    102926 non-null  int64  
 2   대여소명     102926 non-null  object 
 3   대여구분코드   102926 non-null  object 
 4   성별       90886 non-null   object 
 5   연령대코드    102926 non-null  object 
 6   이용건수     102926 non-null  int64  
 7   운동량      102926 non-null  object 
 8   탄소량      102926 non-null  object 
 9   이동거리(M)  102926 non-null  float64
 10  이용시간(분)  102926 non-null  int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 8.6+ MB


In [545]:
Data3.describe()

,대여소번호,이용건수,이동거리(M),이용시간(분)
count,102926.000000,102926.000000,1.029260e+05,102926.000000
mean,2045.927715,17.748149,3.118221e+04,345.388192
std,1479.395437,33.310255,5.534551e+04,607.036734
min,3.000000,1.000000,0.000000e+00,0.000000
25%,843.000000,2.000000,3.442557e+03,42.000000
50%,1729.000000,6.000000,1.128417e+04,127.000000
75%,3121.000000,18.000000,3.417372e+04,374.000000
max,99999.000000,1003.000000,1.233268e+06,17190.000000


In [546]:
Data1.head()

,대여소\r\n번호,보관소(대여소)명,소재지(위치),Unnamed: 3,Unnamed: 4,Unnamed: 5,설치\r\n시기,설치형태,Unnamed: 8,운영\r\n방식
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCD,QR,NaN
1,NaN,NaN,자치구,상세주소,위도,경도,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,거치\r\n대수,거치\r\n대수,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,301.0,경복궁역 7번출구 앞,종로구,서울특별시 종로구 사직로 지하130,37.575794,126.971451,2015-10-07,16,NaN,LCD


In [547]:
#컬럼명 변경
Data1.columns = ['대여소번호', '대여소명', '자치구', '상세주소', '위도', '경도', '설치시기', '거치대수lcd', '거치대수qr', '운영방식']

In [548]:
#불필요 칼럼 제거
Data1 = Data1.drop(['상세주소', '위도', '경도'], axis = 1, inplace = False)
Data1.head()

,대여소번호,대여소명,자치구,설치시기,거치대수lcd,거치대수qr,운영방식
0,NaN,NaN,NaN,NaN,LCD,QR,NaN
1,NaN,NaN,자치구,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,거치\r\n대수,거치\r\n대수,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,301.0,경복궁역 7번출구 앞,종로구,2015-10-07,16,NaN,LCD


In [549]:
#결측치 처리
Data1['거치대수lcd'] = Data1['거치대수lcd'].fillna(0)
Data1['거치대수qr'] = Data1['거치대수qr'].fillna(0)
Data1.head()

,대여소번호,대여소명,자치구,설치시기,거치대수lcd,거치대수qr,운영방식
0,NaN,NaN,NaN,NaN,LCD,QR,NaN
1,NaN,NaN,자치구,NaN,0,0,NaN
2,NaN,NaN,NaN,NaN,거치\r\n대수,거치\r\n대수,NaN
3,NaN,NaN,NaN,NaN,0,0,NaN
4,301.0,경복궁역 7번출구 앞,종로구,2015-10-07,16,0,LCD


In [550]:
Data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2161 entries, 0 to 2160
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   대여소번호    2154 non-null   float64
 1   대여소명     2154 non-null   object 
 2   자치구      2155 non-null   object 
 3   설치시기     2152 non-null   object 
 4   거치대수lcd  2161 non-null   object 
 5   거치대수qr   2161 non-null   object 
 6   운영방식     2154 non-null   object 
dtypes: float64(1), object(6)
memory usage: 118.3+ KB


In [551]:
Data1[Data1['대여소번호'].isnull()]

,대여소번호,대여소명,자치구,설치시기,거치대수lcd,거치대수qr,운영방식
0,NaN,NaN,NaN,NaN,LCD,QR,NaN
1,NaN,NaN,자치구,NaN,0,0,NaN
2,NaN,NaN,NaN,NaN,거치\r\n대수,거치\r\n대수,NaN
3,NaN,NaN,NaN,NaN,0,0,NaN
2158,NaN,NaN,NaN,NaN,0,0,NaN
2159,NaN,NaN,NaN,NaN,0,0,NaN
2160,NaN,NaN,NaN,NaN,0,0,NaN


In [553]:
# 대여소번호 == NaN -> 필요없는 데이터
# 결측치 삭제
Data1 = Data1.dropna(subset=['대여소번호'])
Data1.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2154 entries, 4 to 2157
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   대여소번호    2154 non-null   float64
 1   대여소명     2154 non-null   object 
 2   자치구      2154 non-null   object 
 3   설치시기     2152 non-null   object 
 4   거치대수lcd  2154 non-null   object 
 5   거치대수qr   2154 non-null   object 
 6   운영방식     2154 non-null   object 
dtypes: float64(1), object(6)
memory usage: 134.6+ KB


In [554]:
#대여소명 정리 (숫자 제거)
#대여소명 구성 -> 숫자+ . +대여소이름
Data3['대여소명'].str.split('.', expand = True).head() #expand 새로운 데이터 프레임 생성
Data3['대여소명_new'] = Data3['대여소명'].str.split('.', expand = True)[1]

In [555]:
Data3.head(10)

,대여일자,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리(M),이용시간(분),대여소명_new
0,2021-12,3,중랑센터,일일(회원),M,AGE_008,2,31.94,0.25,1090.00,23,None
1,2021-12,3,중랑센터,정기,M,AGE_004,3,66.29,0.49,2130.00,22,None
2,2021-12,3,중랑센터,정기,M,AGE_005,3,57.35,0.50,2161.63,31,None
3,2021-12,3,중랑센터,정기,M,AGE_008,1,237.99,2.02,8710.00,38,None
4,2021-12,5,상암센터 정비실,일일(회원),M,AGE_004,13,1214.72,10.45,45109.52,356,None
5,2021-12,102,102. 망원역 1번출구 앞,단체,M,AGE_001,2,76.50,0.69,2972.06,19,망원역 1번출구 앞
6,2021-12,102,102. 망원역 1번출구 앞,단체,M,AGE_002,3,0.00,0.00,0.00,158,망원역 1번출구 앞
7,2021-12,102,102. 망원역 1번출구 앞,일일(비회원),\N,AGE_008,8,338.05,3.05,13133.04,407,망원역 1번출구 앞
8,2021-12,102,102. 망원역 1번출구 앞,일일(회원),\N,AGE_001,3,187.29,1.87,8061.40,57,망원역 1번출구 앞
9,2021-12,102,102. 망원역 1번출구 앞,일일(회원),\N,AGE_002,95,5355.55,51.80,223391.73,2960,망원역 1번출구 앞


In [556]:
# 성별 \N 을 None으로 바꿈
# 결측값을 'unknown'으로 대체
Data3['성별'] = np.where(Data3['성별'] =='\\N', None, Data3['성별'])
Data3['성별'] = Data3['성별'].fillna('unknown')
Data3.head(10)

,대여일자,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리(M),이용시간(분),대여소명_new
0,2021-12,3,중랑센터,일일(회원),M,AGE_008,2,31.94,0.25,1090.00,23,None
1,2021-12,3,중랑센터,정기,M,AGE_004,3,66.29,0.49,2130.00,22,None
2,2021-12,3,중랑센터,정기,M,AGE_005,3,57.35,0.50,2161.63,31,None
3,2021-12,3,중랑센터,정기,M,AGE_008,1,237.99,2.02,8710.00,38,None
4,2021-12,5,상암센터 정비실,일일(회원),M,AGE_004,13,1214.72,10.45,45109.52,356,None
5,2021-12,102,102. 망원역 1번출구 앞,단체,M,AGE_001,2,76.50,0.69,2972.06,19,망원역 1번출구 앞
6,2021-12,102,102. 망원역 1번출구 앞,단체,M,AGE_002,3,0.00,0.00,0.00,158,망원역 1번출구 앞
7,2021-12,102,102. 망원역 1번출구 앞,일일(비회원),unknown,AGE_008,8,338.05,3.05,13133.04,407,망원역 1번출구 앞
8,2021-12,102,102. 망원역 1번출구 앞,일일(회원),unknown,AGE_001,3,187.29,1.87,8061.40,57,망원역 1번출구 앞
9,2021-12,102,102. 망원역 1번출구 앞,일일(회원),unknown,AGE_002,95,5355.55,51.80,223391.73,2960,망원역 1번출구 앞


In [557]:
#정비센터-> 대여소해당X 대여소명 결측값
Data3[Data3['대여소명_new'].isnull()]

,대여일자,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리(M),이용시간(분),대여소명_new
0,2021-12,3,중랑센터,일일(회원),M,AGE_008,2,31.94,0.25,1090.00,23,None
1,2021-12,3,중랑센터,정기,M,AGE_004,3,66.29,0.49,2130.00,22,None
2,2021-12,3,중랑센터,정기,M,AGE_005,3,57.35,0.50,2161.63,31,None
3,2021-12,3,중랑센터,정기,M,AGE_008,1,237.99,2.02,8710.00,38,None
4,2021-12,5,상암센터 정비실,일일(회원),M,AGE_004,13,1214.72,10.45,45109.52,356,None
102885,2021-12,9985,천왕센터,정기,M,AGE_004,1,25.98,0.19,820.00,7,None
102886,2021-12,9992,사대문센터,일일(회원),M,AGE_004,10,970.00,8.36,36021.44,310,None
102892,2021-12,9999,상담센터,단체,unknown,AGE_003,1,0.00,0.00,0.00,14,None
102893,2021-12,9999,상담센터,단체,F,AGE_002,1,70.91,0.88,3810.00,33,None
102894,2021-12,9999,상담센터,단체,F,AGE_003,1,75.03,0.81,3508.63,49,None


In [558]:
# 대여소가 정비센터 -> 삭제
Data3 = Data3.dropna(subset=['대여소명_new'])
Data3.head(10)


,대여일자,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리(M),이용시간(분),대여소명_new
5,2021-12,102,102. 망원역 1번출구 앞,단체,M,AGE_001,2,76.50,0.69,2972.06,19,망원역 1번출구 앞
6,2021-12,102,102. 망원역 1번출구 앞,단체,M,AGE_002,3,0.00,0.00,0.00,158,망원역 1번출구 앞
7,2021-12,102,102. 망원역 1번출구 앞,일일(비회원),unknown,AGE_008,8,338.05,3.05,13133.04,407,망원역 1번출구 앞
8,2021-12,102,102. 망원역 1번출구 앞,일일(회원),unknown,AGE_001,3,187.29,1.87,8061.40,57,망원역 1번출구 앞
9,2021-12,102,102. 망원역 1번출구 앞,일일(회원),unknown,AGE_002,95,5355.55,51.80,223391.73,2960,망원역 1번출구 앞
10,2021-12,102,102. 망원역 1번출구 앞,일일(회원),unknown,AGE_003,25,1116.58,8.63,37173.74,430,망원역 1번출구 앞
11,2021-12,102,102. 망원역 1번출구 앞,일일(회원),unknown,AGE_004,5,489.01,5.23,22563.37,203,망원역 1번출구 앞
12,2021-12,102,102. 망원역 1번출구 앞,일일(회원),unknown,AGE_008,14,1084.35,10.06,43408.97,290,망원역 1번출구 앞
13,2021-12,102,102. 망원역 1번출구 앞,일일(회원),unknown,AGE_002,4,108.51,1.01,4356.99,112,망원역 1번출구 앞
14,2021-12,102,102. 망원역 1번출구 앞,일일(회원),F,AGE_001,5,143.19,1.31,5640.27,133,망원역 1번출구 앞


In [559]:
#Data2도 Data3와 마찬가지로 대여소명을 번호와 구분지어서 생성
Data2['대여소번호'] = Data2['대여소 명'].str.split(".", expand = True)[0]
Data2['대여소명_new'] = Data2['대여소 명'].str.split(".", expand = True)[1]

In [560]:
Data2.head(10)

,대여소 그룹,대여소 명,대여 일자 / 월,대여 건수,대여소번호,대여소명_new
0,강남구,2301. 현대고등학교 건너편,202107,2545,2301,현대고등학교 건너편
1,강남구,2302. 교보타워 버스정류장(신논현역 3번출구 후면),202107,1176,2302,교보타워 버스정류장(신논현역 3번출구 후면)
2,강남구,2303. 논현역 7번출구,202107,1467,2303,논현역 7번출구
3,강남구,2304. 신영 ROYAL PALACE 앞,202107,349,2304,신영 ROYAL PALACE 앞
4,강남구,2305. MCM 본사 직영점 앞,202107,341,2305,MCM 본사 직영점 앞
5,강남구,2306. 압구정역 2번 출구 옆,202107,1265,2306,압구정역 2번 출구 옆
6,강남구,2307. 압구정 한양 3차 아파트,202107,383,2307,압구정 한양 3차 아파트
7,강남구,2308. 압구정파출소 앞,202107,673,2308,압구정파출소 앞
8,강남구,2309. 청담역(우리들병원 앞),202107,418,2309,청담역(우리들병원 앞)
9,강남구,2310. 청담동 맥도날드 옆(위치),202107,410,2310,청담동 맥도날드 옆(위치)


In [561]:
#정비센터-> 대여소해당X 대여소명 결측값
Data2[Data2['대여소명_new'].isnull()]

,대여소 그룹,대여소 명,대여 일자 / 월,대여 건수,대여소번호,대여소명_new
2198,정비센터,상담센터,202107,138,상담센터,None
2199,정비센터,상암센터 정비실,202107,8,상암센터 정비실,None
2200,정비센터,위트콤,202107,8,위트콤,None
2201,정비센터,중랑센터,202107,24,중랑센터,None
2202,정비센터,천왕센터,202107,1,천왕센터,None
2203,정비센터,천호센터,202107,0,천호센터,None
2204,정비센터,훈련원센터,202107,3,훈련원센터,None
2477,그룹명 없음,대여소명 없음,202108,0,대여소명 없음,None
4694,정비센터,도봉정비센터,202108,7,도봉정비센터,None
4695,정비센터,상담센터,202108,175,상담센터,None


In [562]:
NewData2 = Data2.dropna(subset=['대여소명_new'])